In [2]:
import pandas as pd
import numpy as np

In [3]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_train = pd.read_csv('C:/Users/DELL/Downloads/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('C:/Users/DELL/Downloads/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [8]:
ratings_train.head()

,user_id,movie_id,rating,unix_timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [19]:
n_users = ratings_train.user_id.unique().shape[0]
n_items = ratings_train.movie_id.unique().shape[0]


max_user_id = max(ratings_train['user_id'])
max_item_id = max(ratings_train['movie_id'])

print(n_users, n_items)
print(max_user_id, max_item_id)

943 1680
943 1682


In [25]:
data_matrix = np.zeros((n_users, max_item_id)) 
for line in ratings_train.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

In [26]:
data_matrix.shape

(943, 1682)

In [27]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [30]:
print(user_similarity.shape)
print(item_similarity.shape)

(943, 943)
(1682, 1682)


In [63]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])   # a[:, np.newaxis] gives transpose of a
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
        
    return pred

In [70]:
user_prediction = predict(data_matrix, item_similarity, type='item')

In [76]:
sorted(user_prediction[0], reverse=True)[0:30]

[0.5876810604429497,
 0.5853713629285401,
 0.5838293104207221,
 0.5836933873132876,
 0.5834056514162491,
 0.5834056514162491,
 0.5821927515351616,
 0.5821927515351616,
 0.5817111972929514,
 0.5815507222319174,
 0.5805564635479896,
 0.5804322991010413,
 0.5797711335807268,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605,
 0.5791359790166605]

In [77]:
user_prediction.shape

(943, 1682)

In [75]:
user_prediction[0]

array([0.43389504, 0.46234738, 0.48689667, ..., 0.56539575, 0.55089193,
       0.54418388])